first write function that gets scores of all stores

get top 100 and make into portfolio (just a list)

Get Portfolio Stats (load periodic data and add up)
-own fortnightly E[r] and sd(r)
-calculate E(r) / sd(r)

then write loop for alphas